In [1]:
import os
import re
import subprocess
from pathlib import Path
from io import BytesIO

import pandas as pd
from Bio import SeqIO

In [2]:
database = '/media/GenomicResearch/Issue/NCBI/Blast_Database/16S_ribosomal_RNA/16S_ribosomal_RNA.fasta'

In [3]:
taxmap = {record.id: re.search('\w+ \w+', record.description.split(None, 1)[-1]).group() for record in SeqIO.parse(database, 'fasta')}

In [4]:
def count_reads_number(query):
    process = subprocess.run(['nanoq', '-s', '-i', query, '-l', '1200', '-m', '1800'], stderr=subprocess.PIPE)
    num_reads = int(process.stderr.decode().split()[0])
    return num_reads


def sequence_mapping(query, database, threads):
    nanoq_process = subprocess.Popen(
        ['nanoq', '-i', query, '-l', '1200', '-m', '1800'],
        stdout=subprocess.PIPE
    )
    output = subprocess.check_output(
        ['minimap2', '-cx', 'map-ont', '-z', '70', '-K', '8G', '-t', str(threads), database, '-'],
        stdin=nanoq_process.stdout
    )
    return output


def load_paf(stream):
    names = [
        "Query", "Q_length", "Q_start", "Q_end", "Strand", "op", "T_length", "T_start",
        "T_end", "N_res_matches", "Align_block", "MapQ", "NM", "ms", "AS", "nn", "P_S",
    ]
    return pd.read_csv(BytesIO(stream), sep='\t', usecols=range(0, 17), names=names)


def workflow(query, database, taxmap, output, threads=16):
    num_reads = count_reads_number(query)
    mapping_output = sequence_mapping(query, database, threads)
    df = load_paf(mapping_output)
    df = df.sort_values('AS', key=lambda x: x.str.split(':').str[-1].astype(int), ascending=False).drop_duplicates('Query')
    df = df[df['Align_block']>1000]
    df['organism_name'] = df['op'].map(taxmap)
    abundance = df.groupby('organism_name').size().sort_values(ascending=False).div(num_reads).mul(100).round(2)
    abundance.name = 'abundance'
    abundance.to_csv(output, sep='\t')

In [6]:
dirpath = Path('/media/Central_Lab_Storage/MinION/mNGS/20220111_16SA/fastq')
outpath = Path('/media/Central_Lab_Storage/MinION/mNGS/20220111_16SA/mapping')

for i in dirpath.iterdir():
    output = outpath/i.name.replace('.fastq.gz', '.txt')
    workflow(i, database, taxmap, output, 64)